<a href="https://colab.research.google.com/github/KorStats/skku/blob/main/%ED%86%B5%ED%95%A9/%EC%9C%B5%ED%95%A91)_EDA_%EA%B3%BC%EC%A0%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 25.5 MB/s eta 0:00:00


In [2]:
import pickle
import random
from konlpy.tag import Okt

# 동의어 사전 로드
def load_synonym_dict(pickle_file):
    with open(pickle_file, 'rb') as f:
        synonym_dict = pickle.load(f)
    return synonym_dict

# Random Swap (랜덤 단어 교환)
def random_swap(words, n=1):
    words = words.copy()
    for _ in range(n):
        idx1, idx2 = random.sample(range(len(words)), 2)
        words[idx1], words[idx2] = words[idx2], words[idx1]
    return words

# Random Insertion (랜덤 단어 삽입)
def random_insertion(words, synonym_dict, n=1):
    words = words.copy()
    for _ in range(n):
        idx = random.randint(0, len(words) - 1)
        word = words[idx]
        if word in synonym_dict:
            words.insert(idx, random.choice(synonym_dict[word]))
    return words

# Random Delete (랜덤 단어 삭제)
def random_delete(words, p=0.1):
    if len(words) == 1:
        return words
    return [word for word in words if random.random() > p]

# Synonym Replacement (동의어 대체)
def synonym_replacement(words, synonym_dict, n=1):
    words = words.copy()
    for _ in range(n):
        idx = random.randint(0, len(words) - 1)
        word = words[idx]
        if word in synonym_dict:
            words[idx] = random.choice(synonym_dict[word])
    return words

# 데이터 증강 실행 함수
#def augment_text(text, synonym_dict, swap_n=1, insert_n=1, delete_p=0.1, replace_n=1):
#    okt = Okt()
#    words = okt.morphs(text)  # Okt를 이용한 토큰화
#    words = random_swap(words, swap_n)
#    words = random_insertion(words, synonym_dict, insert_n)
#    words = random_delete(words, delete_p)
#    words = synonym_replacement(words, synonym_dict, replace_n)
#    return ' '.join(words)

In [3]:
# 동의어 사전 로드
synonym_dict = load_synonym_dict('/content/wordnet.pickle')

In [4]:
# 데이터 증강 실행 함수
def augment_text(text, synonym_dict, swap_n=0, insert_n=0, delete_p=0, replace_n=0):
    okt = Okt()
    words = okt.morphs(text)  # Okt를 이용한 토큰화

    if swap_n > 0:
        words = random_swap(words, swap_n)
    if insert_n > 0:
        words = random_insertion(words, synonym_dict, insert_n)
    if delete_p > 0:
        words = random_delete(words, delete_p)
    if replace_n > 0:
        words = synonym_replacement(words, synonym_dict, replace_n)

    return ' '.join(words)

# 데이터불러오기

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'   이전   졸업논문


In [6]:
import pandas as pd
df= pd.read_csv('/content/gdrive/MyDrive/졸업논문/NSMC2_0.02_증분대상_100개_융합1.csv', encoding='utf-8-sig')

In [7]:
df.head(3)

,id,ori_doc,label,ext,abs,bt,document
0,9983364.0,본 영화중 가장 특이한 분위기의 영화같다. 내용도 독특하고 억지스러운부분도 있고 맘...,0.0,본 영화중 가장 특이한 분위기의 영화같다. 쓸데없이 스토리 전개는 느릿하고 반복되고.,졸리캐릭터는 가엾은 기구한운명처럼 보이나 호감이 가진않음.,"그것은 가장 특이한 분위기를 가진 영화와 같다. 내용은 독특하고 강제적이며, 나는 ...",본 영화중 가장 특이한 분위기의 영화같다. 쓸데없이 스토리 전개는 느릿하고 반복되고.
1,9325100.0,점점 내용이 이상해지는것같다 소라가 백연희딸이라더만 이젠 이화영은 정병국딸이란다 막...,0.0,점점 내용이 이상해지는것같다 소라가 백연희딸이라더만 이젠 이화영은 정병국딸이란다 막...,소라가 백연희딸이라더만 이젠 이화영은 이화영은 정병국딸이란다 막장드라마라는건 알고있...,가장 특이한 대기 영화와 같습니다. 내용은 독특하고 강요 당하고 나는 그것을 좋아하...,점점 내용이 이상해지는것같다 소라가 백연희딸이라더만 이젠 이화영은 정병국딸이란다 막...
2,8236827.0,지섭이한테 얘기안하고 끝나는거보니 또 둘러대고 넘어가겟네요~ 일단 이렇게 넘어가고 ...,0.0,지섭이한테 얘기안하고 끝나는거보니 또 둘러대고 넘어가겟네요~ 일단 이렇게 넘어가고 ...,지섭이한테 얘기안하고 끝나는거보니 또 둘러대고 넘어가겟네요~ 일단 이렇게 넘어가고 ...,그것은 가장 특이한 분위기를 가진 영화와 같습니다. 내용은 독특하고 마음에 들지 않...,지섭이한테 얘기안하고 끝나는거보니 또 둘러대고 넘어가겟네요~ 일단 이렇게 넘어가고 ...


In [8]:
# 증강 적용
df['text_swap'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=2))
df['text_insert'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=2))
df['text_delete'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.2))
df['text_replace'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=2))

In [9]:
df.to_csv('NSMC2_융합1_eda.csv', encoding='utf-8-sig')